In [181]:
from __future__ import print_function

import os;
import shutil;
import codecs;
import zipfile;
import json;
import sqlite3;

import re;
import requests
from lxml import html
from string import ascii_uppercase;
import chardet;
import random;
#from BeautifulSoup import BeautifulSoup as bs

#baixar simultaneo
from multiprocessing import Pool;

In [43]:
from os.path import expanduser
home = expanduser("~")

In [44]:
baseAddress = 'https://www.letras.mus.br';

# Utility Functions

In [45]:
def slugify(value):
    """
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens.
    """
    import unicodedata
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore')
    value = re.sub('[^\w\s-]', '', value).strip().lower()
    value = re.sub('[-\s]+', '-', value)
    return value;

In [47]:
#use chardet to detect the actual encoding of the content and than encode it.
def decode(content):
    enconding = chardet.detect(content)['encoding']; #converting to the right encoding
    content = content.decode(enconding, errors='ignore');
    return content;

# Web Scrapping Related Functions

In [64]:
def listArtistByLetter(letter):
    page = requests.get(baseAddress+'/letra/'+letter+'/artistas.html')
    tree = html.fromstring(page.content.decode(page.encoding));
    return [
        a.attrib['href'] for a in tree.xpath('//div[@class="artistas-a js-alphabet-cnt"]/ul/li//a')
    ];

In [66]:
# retrive only one song lyrics
def getLyrics(id):
    page = requests.get(baseAddress+id);
    return page.content.decode(page.encoding);

In [188]:
def htmlRemoveNotContentTags(htmlString):
    tree = html.fromstring(htmlString);
    for t in ["//script",
              "//noscript",
              "//style",
              "//link",
              '//noscript',
              "//footer",
              "//comment()",
              "//*[@rel='dns-prefetch']",
              "//*[@rel='alternate']",
              "//b[not(node())]",  #empty nodes
              "//div[not(text()) and not(node())]",
              "//span[not(text()) and not(node())]",
              "//hr[not(text()) and not(node())]",
              "//i[not(text()) and not(node())]"]:
        for  bad in tree.xpath("//"+t):
            #bad.clear();
            bad.getparent().remove(bad);

    return html.tostring(tree, pretty_print=False);

# Reading all artists into array to futher processing

## Reading only the most visited artist (probably it contains the weekly report, not the all times report) and artist in the top 1000 musics

In [69]:
maisAcessados = baseAddress+'/mais-acessadas/';
maisAcessados = requests.get(maisAcessados);
maisAcessados = maisAcessados.content.decode(maisAcessados.encoding)
maisAcessados = html.fromstring(maisAcessados);
pt1 = set(re.sub(r'(/.*?)/.*',r'\1/',a.attrib['href']) for a in maisAcessados.xpath('//*[contains(@class,"top-list_mus")]/li//a'));
pt2 = set(a.attrib['href'] for a in maisAcessados.xpath('//*[@class="top-list_art"]/li//a'));
maisAcessados = list(pt1.union(pt2));
del pt1;
del pt2;

In [70]:
len(maisAcessados)

482

In [116]:
maisAcessados = [w
                for w in maisAcessados
                     if
                    'ministerio' not in w
                    and 'comunidade' not in w
                    and 'catolicas' not in w
                    and 'padre' not in w
                    and 'pastor' not in w
                    and 'louvor' not in w
                    and 'igreja' not in w
                ]

In [117]:
maisAcessados = sorted(maisAcessados)

In [118]:
len(maisAcessados)

468

In [82]:
conn = sqlite3.connect('lyrics.db')
c = conn.cursor()

In [85]:
# Create table
c.execute('DROP table Lyrics;')
c.execute('''
CREATE TABLE Lyrics(
    genre text,
    artist text,
    title text,
    exbibitions  int,
    contributor text,
    compositor text,
    songHtml text

)
''');

In [ ]:
def list downloadedSongs(conn,artist):
    cursor = c.execute('select distinct title from {tn} where artist = \'{artist}\''.format(tn='Lyrics',artist=artist));
    dowloaded =[];
    for row in cursor;
        download.append(row[0]);
    return downloaded;

In [97]:
c.execute('''
    INSERT INTO {tn} (genre, artist,title,exbibitions,contributor,compositor,songHtml)
    VALUES('{genre}', '{artist}','{title}','{exbibitions}','{contributor}','{compositor}','{songHtml}')
    '''.format(tn='Lyrics',genre="teste1",artist="teste2",title='title',exbibitions=4, contributor='contributor',compositor='compositor',songHtml='songHtml'))

In [153]:
def listSongsFromArtist(artist):
    page = requests.get(baseAddress+artist)
    tree = html.fromstring(page.content.decode(page.encoding));
    songList = [a for a in tree.xpath('//div[@class="cnt-list--alp"]/li//a')] + [a for a in tree.xpath('//*[@id="cnt_top"]//li//a')]
    songList = {a.attrib['href'].replace(artist,''):a.text_content() for a in songList};
    return songList;  

In [191]:
def parseSong(songFile):
    tree = html.fromstring(songFile);
    
    songText = tree.xpath("//*[contains(@class,'cnt-letra')]/article");
    if len(songText) == 0:
        return None;
    songText = html.tostring(songText[0],pretty_print=True);
    songText = songText.replace("<br>","\n");
    
    breadCrumb = tree.xpath("//*[@id='breadcrumb']//*[@itemprop='name']//text()");
    songTile = tree.xpath("//*[@id='js-lyric-cnt']//h1/text()")[0].strip();
    exbibitions = tree.xpath("//*[@class='cnt-info_exib']/b/text()");
    if len(exbibitions)>0:
        exbibitions = float(re.sub(r'\D','', exbibitions[0]))  
        
    contributor = tree.xpath("//*[@class='letra-info_user']/a/@href");
    if len(contributor)>0:
        contributor = contributor[0].strip();

    
    song ={
        "genre":breadCrumb[1].strip(),
        "artist":breadCrumb[2].strip(),
        'title':songTile,
        'songHtml':songText,
        'exbibitions':exbibitions,
        'contributor':contributor,
        'compositor':tree.xpath("//*[@class='letra-info_comp']/text()")[0].replace(u"Composi\xe7\xe3o:","").strip(),        
    }
    return song;  

In [161]:
source = getLyrics('/calvin-harris/'+list(temp.keys())[0])

In [192]:
content = htmlRemoveNotContentTags(source); 
parseSong(content)

TypeError: a bytes-like object is required, not 'str'

In [ ]:
def downloadArtist(conn,artist):
    dowloaded = downloadedSongs(conn, artist);
    songsFromWeb = listSongsFromArtist(artist);
    
    for s in songsFromWeb:
        if songsFromWeb[s] not in dowloaded:
            source = getLyrics(artist+s);
            source = htmlRemoveNotContentTags(source); 
    

In [ ]:
        "genre":breadCrumb[1].strip(),
        "artist":breadCrumb[2].strip(),
        'title':songTile,
        'songHtml':songText,
        'exbibitions':exbibitions,
        'contributor':contributor,
        'compositor':tree.xpath("//*[@class='letra-info_comp']/text()")[0].replace(u"Composi\xe7\xe3o:","").strip(),       

In [99]:
cursor = c.execute("SELECT * from Lyrics")
for row in cursor:
    print()
    for idx, col in enumerate(cursor.description):
        print(col[0],'=',row[idx])



genre = teste1
artist = teste2
title = title
exbibitions = 4
contributor = contributor
compositor = compositor
songHtml = songHtml

genre = teste1
artist = teste2
title = title
exbibitions = 4
contributor = contributor
compositor = compositor
songHtml = songHtml


In [15]:
def downloadArtist(currentArtist):
    artistDir= home + "/letras"+currentArtist[0:-1]+".zip";
    if os.path.exists(artistDir):
        return;
    
    songList = listSongsFromArtist(currentArtist);
    
    if len(songList) <3:
        print ("less than 3 songs %s"%currentArtist);
        return;
    
    #checking updates
    if os.path.exists(artistDir):
        with zipfile.ZipFile(artistDir,'r') as thezip:
            stored = thezip.namelist();
            for fileName in stored:
                if fileName in songList:
                    del songList[fileName];
            
            if len(songList) >0:
                print ("Update %s"%currentArtist)
                temp = zipfile.ZipFile(artistDir+'temp.zip','w');
                #old files
                stored = thezip.namelist();
                for fname in stored:
                    temp.write(fname, thezip.read(fname));
                  
                #new files
                for fname in songList:
                    content = decode(getLyrics(songList[fname]));
                    content = htmlRemoveNotContentTags(content);                        
                    temp.writestr(fname, content.encode("UTF-8"));
                temp.close();
                
                shutil.move(artistDir+'temp.zip', artistDir);
    else:
        
        thezip = zipfile.ZipFile(artistDir,'w');
        inZip = []; #adicional check for duplicated files.
        for fileName in songList:
            if fileName in inZip:
                continue;
            
            content = decode(getLyrics(songList[fileName]));
            content = htmlRemoveNotContentTags(content);
    
            thezip.writestr(fileName, content.encode("UTF-8"));
            inZip.append(fileName);
        
        thezip.close();

### maisAcessadas

In [16]:
print ("Faltando %d artistas para ler " % len(maisAcessados))
processados = [];
'''
for i, currentArtist in enumerate(maisAcessados):
    currentArtist = currentArtist.strip();
    print (" %d %s "%(i, currentArtist));         
    downloadArtist(currentArtist);

maisAcessados = list(set(maisAcessados) - set(processados));
'''
def process(e):
    i, currentArtist = e;
    try:
        print (" %d %s "%(i, currentArtist));         
        downloadArtist(currentArtist);
    except Exception as inst:
        print ("Error reading %s  | %s"%(currentArtist, inst));
    
    
p = Pool(5);
p.map(process, enumerate(maisAcessados));

Faltando 471 artistas para ler 
 48 /barao-vermelho/ 
 72 /cancao-louvor/ 
 0 /1kilo/ 
 24 /anderson-freire/ 
 96 /cnco/ 
 1 /3030/ 
 49 /barbara-dias/ 
 73 /cancao-nova/ 
 25 /andra-day/ 
 97 /coldplay/ 
 2 /4-non-blondes/ 
 50 /bazzi/ 
 74 /cantigas-populares/ 
 98 /coral-kemuel/ 
 26 /andre-aquino/ 
 3 /5-seco/ 
 51 /bebe-rexha/ 
 75 /capital-inicial/ 
 99 /corinhos-evangelicos/ 
 27 /angra/ 
 52 /bee-gees/ 
 100 /costa-gold/ 
 4 /adele/ 
 28 /anitta/ 
 76 /carolina-deslandes/ 
 53 /beija-flor-rj/ 
 5 /adoracao-e-adoradores/ 
 101 /cristiano-araujo/ 
 29 /anjos-de-resgate-musicas/ 
 77 /cassia-eller/ 
 6 /aerosmith/ 
 102 /cristina-mel/ 
 54 /belchior/ 
 30 /antonia-gomes/ 
 78 /cassiane/ 
 55 /belo/ 
 103 /cynthia-luz/ 
 7 /ah-mr-dan/ 
 31 /arctic-monkeys/ 
 79 /catolicas/ 
 56 /benito-di-paula/ 
 8 /alan-walker/ 
 104 /damares/ 
 32 /ari-acustico/ 
 57 /berimbolo/ 
 80 /cazuza/ 
 9 /alceu-valenca/ 
 105 /dani-russo/ 
 33 /ariana-grande/ 
 81 /cecilia-krull/ 
 34 /armandinho/ 
 10 

# Limpeza do HTML

In [17]:
from lxml import etree
import glob, shutil;
import json;

In [18]:
def parseSong(songFile):
    tree = html.fromstring(songFile);
    
    songText = tree.xpath("//*[contains(@class,'cnt-letra')]/article");
    if len(songText) == 0:
        return None;
    songText = html.tostring(songText[0],pretty_print=True);
    songText = songText.replace("<br>","\n");
    
    breadCrumb = tree.xpath("//*[@id='breadcrumb']//*[@itemprop='name']//text()");
    songTile = tree.xpath("//*[@id='js-lyric-cnt']//h1/text()")[0].strip();
    exbibitions = tree.xpath("//*[@class='cnt-info_exib']/b/text()");
    if len(exbibitions)>0:
        exbibitions = float(re.sub(r'\D','', exbibitions[0]))  
        
    contributor = tree.xpath("//*[@class='letra-info_user']/a/@href");
    if len(contributor)>0:
        contributor = contributor[0].strip();

    
    song ={
        "genre":breadCrumb[1].strip(),
        "artist":breadCrumb[2].strip(),
        'title':songTile,
        'songHtml':songText,
        'exbibitions':exbibitions,
        'contributor':contributor,
        'compositor':tree.xpath("//*[@class='letra-info_comp']/text()")[0].replace(u"Composi\xe7\xe3o:","").strip(),        
    }
    return song;  

In [19]:
songs= [];

for zippath in sorted(glob.glob(r"/Users/joseeleandrocustodio/letras/*.zip")):
    print(zippath);
    try:
        with zipfile.ZipFile(zippath,"r") as thezip:
            for fname in thezip.namelist():
                content = thezip.read(fname);
                s = parseSong(content.decode("utf-8"));
                if s:
                    songs.append(s);
    except IndexError as e:
        print ("Error reading file:"+zippath +" song:"+fname);
        print (e)
        break;
    
with codecs.open(r'/Users/joseeleandrocustodio/letras/songs.json', mode='w') as outfile:
    json.dump(songs, outfile);

/Users/joseeleandrocustodio/letras/1kilo.zip
/Users/joseeleandrocustodio/letras/3-doors-down.zip
/Users/joseeleandrocustodio/letras/3030.zip
/Users/joseeleandrocustodio/letras/4-non-blondes.zip
/Users/joseeleandrocustodio/letras/5-seco.zip
/Users/joseeleandrocustodio/letras/a-banda-mais-bonita-da-cidade.zip
/Users/joseeleandrocustodio/letras/adele.zip
/Users/joseeleandrocustodio/letras/adl-mcs.zip
/Users/joseeleandrocustodio/letras/adoracao-e-adoradores.zip
/Users/joseeleandrocustodio/letras/aerosmith.zip
/Users/joseeleandrocustodio/letras/ah-mr-dan.zip
/Users/joseeleandrocustodio/letras/alan-walker.zip
/Users/joseeleandrocustodio/letras/alceu-valenca.zip
/Users/joseeleandrocustodio/letras/alejandro-sanz.zip
/Users/joseeleandrocustodio/letras/alessandro-vilas-boas.zip
/Users/joseeleandrocustodio/letras/alicia-keys.zip
/Users/joseeleandrocustodio/letras/aline-barros.zip
/Users/joseeleandrocustodio/letras/alisson-e-neide.zip
/Users/joseeleandrocustodio/letras/almir-sater.zip
/Users/josee

In [20]:
def cleanZip(zippath):
    with zipfile.ZipFile(zippath,"r") as thezip:
        with zipfile.ZipFile(zippath+".clean.zip","w") as zipclean:
            for fname in thezip.namelist():
                content = thezip.read(fname);
                content = htmlRemoveNotContentTags(content.decode("UTF-8"));
                zipclean.writestr(fname,content);

In [ ]:
for g in glob.glob(r"/Users/joseeleandrocustodio/letras/*.zip"):
    cleanZip(g);


In [ ]:
for g in glob.glob(r"/Users/joseeleandrocustodio/letras/*.clean.zip"):
    shutil.move(g,r"/Users/joseeleandrocustodio/letrasClean")
    
for g in glob.glob(r"/Users/joseeleandrocustodio/letras/*.clean.zip"):
    shutil.move(g,g.replace(".clean.zip",""))    

In [ ]:
lyrics = None;

with codecs.open(home+r'/letras/songs.json', mode='r') as outfile:
    lyrics = json.load(outfile);

In [ ]:
for l in lyrics:
    if l['compositor']:
        comp = l['compositor'];
        comp = comp[0:-2].strip();
        comp = re.sub(r'\s{2,}',' ',comp);
        l['compositor'] = comp;
    if l['contributor']:
        l['contributor'] = re.sub(r'[^\d]','',l['contributor'])
    
    l['songHtml'] = re.sub('\s*\<(\/)?article\>\s*','',l['songHtml'])
    
    if len(l['genre']) < 2:
        l['genre'] = 'desconhecido';

    
with codecs.open(home + r'/letras/songs2.json', mode='w') as outfile:
    json.dump(lyrics, outfile);

In [ ]:
print(re.sub('\s*\<(\/)?article\>\s*','',lyrics[1]['songHtml']))

# rascunhos

In [ ]:
for zippath in sorted(glob.glob(r"/Users/joseeleandrocustodio/letras/*.zip")):
    try:
        with zipfile.ZipFile(zippath) as z:
            if z.testzip() is not None:
                print ("bad zip %s"%zippath)
    except:
        os.remove(zippath)
        print ("bad zip %s"%zippath)